# Class II - An introduction to JuMP

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of JuMP. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) first.

This tutorial doesn't exist in isolation. Some other good resources for learning JuMP are
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.18/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)

Before we start, run the following magic sauce to install the required Julia packages and check that we're good to go.

In [45]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

Activating environment at `~/Documents/dev/tutorial-grid-science-2019/Project.toml`
Excellent! Everything is good to go!


## The basics

First, load the JuMP package into your current environment.

In [46]:
using JuMP

Start building a JuMP model like so,

In [47]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

A JuMP Model
Feasibility problem with:
Variables: 3
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y, z

### What's going on here?
`@variable(model, x)` does four things:
1. it adds an *optimization* variable to the model
2. it creates a *JuMP* variable that acts as a reference to the optimization variable in the model
3. it creates a *Julia* variable `x` that points to the JuMP variable
4. it stores a reference to the JuMP variable in the model with the name `:x`

In [48]:
model = Model()
@variable(model, x >= 1.414)

x

In [49]:
# x is a JuMP variable
typeof(x)

VariableRef

In [50]:
# We can bind the JuMP variable to a different Julia variable and set `x` to something else
y = x
x = 1

@show typeof(y)
@show typeof(x)

y

typeof(y) = VariableRef
typeof(x) = Int64


x

In [51]:
JuMP.lower_bound(y)

1.414

In [52]:
model[:x]

x

In [53]:
model[:x] == y

true

### Other ways to create variables

We can also create arrays of JuMP variables.

In [54]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

4-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [55]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
x

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Base.OneTo(2)
    Dimension 2, Symbol[:A, :B]
And data, a 2×2 Array{VariableRef,2}:
 x[1,A]  x[1,B]
 x[2,A]  x[2,B]

We can also add a condition to filter out some of the variables

In [56]:
model = Model()
@variable(model, x[i = 1:4, j = [:A, :B]; isodd(i)] >= i)
x

JuMP.Containers.SparseAxisArray{VariableRef,2,Tuple{Int64,Symbol}} with 4 entries:
  [3, A]  =  x[3,A]
  [3, B]  =  x[3,B]
  [1, A]  =  x[1,A]
  [1, B]  =  x[1,B]

What if I want to add two variables with the same name?

In [57]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

ErrorException: An object of name x is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

### Quiz Question

What is the value of the following?

In [58]:
JuMP.lower_bound(x)

1.0

### Anonymous variables

So far, we've only added *named* variables. This works great until we want to add two variables with the same name! To work around this isse, we can add *anonymous* variables.

In [59]:
model = Model()

x = @variable(model)
@show JuMP.has_lower_bound(x)

x = @variable(model, lower_bound = 1)
@show JuMP.lower_bound(x)

x = @variable(model, [i = 1:2], lower_bound = i)
@show JuMP.lower_bound.(x)

model

JuMP.has_lower_bound(x) = false
JuMP.lower_bound(x) = 1.0
JuMP.lower_bound.(x) = [1.0, 2.0]


A JuMP Model
Feasibility problem with:
Variables: 4
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

So what's the difference between the *named* and *anonymous* syntax? 

Well, `@variable(model, x >= 2)` is (roughly) equivalent to

In [60]:
model = Model()
x = model[:x] = @variable(model, lower_bound = 2, base_name = "x")
model

A JuMP Model
Feasibility problem with:
Variable: 1
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [61]:
model = Model()
@variable(model, x >= 2)
model

A JuMP Model
Feasibility problem with:
Variable: 1
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

Although uneeded for this tutorial, we can also create binary and integer variables as follows:

In [62]:
model = Model()
@variable(model, x >= 1, Int)
@variable(model, y, Bin)
model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.Integer`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y

### Constraints

Now that we've seen how to create variables, let's look at constraints. Much of the syntax should be familiar.

In [63]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

In [64]:
model[:c_equal_to]

c_equal_to : 2 x + y = 1.0

In [65]:
anonymous_constraint = @constraint(model, [i = 1:2], i * x <= y)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

### Objective Functions

Now let's look at the objective function.

In [66]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [67]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

### Solving a Model

Once we've formulated a model, the next step is to solve it. This requires a solver.

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/v0.19.2/installation/) contains a list of the supported solvers and the types of problems each solver supports.

We're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPK.jl](https://github.com/JuliaOpt/GLPK.jl) package.

The second solver is the COIN-OR [Interior Point OPTimizer (Ipopt)](https://projects.coin-or.org/Ipopt). This solver supports nonlinear programs with continous variables.

Ipopt is available via the [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package.

In [68]:
using GLPK
using Ipopt

There are two ways to add a solver to a JuMP model:

In [69]:
model = Model(GLPK.Optimizer)

# ... or ...

model = Model()
set_optimizer(model, Ipopt.Optimizer)

If you try to solve an unsupported problem type, an error will be thrown:

In [70]:
model = Model(GLPK.Optimizer)
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2)
optimize!(model)

ErrorException: The solver does not support nonlinear problems (i.e., NLobjective and NLconstraint).

That error isn't very nice, but there is an [open JuMP issue](https://github.com/JuliaOpt/JuMP.jl/issues/1996) to resolve this.

In [71]:
set_optimizer(model, Ipopt.Optimizer)
optimize!(model)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

## Interpreting statuses

After solving a model, JuMP can report three different statuses:

- `termination_status(model)` explains why the solver stopped. Common statuses are `OPTIMAL`, `INFEASIBLE`, `DUAL_INFEASIBLE` (i.e., primal is potentially unbounded), and `LOCALLY_SOLVED`.

- `primal_status(model)` explains how to interpret the primal solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

- `dual_status(model)` explains how to interpret the dual solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

## Getting solutions

- Use `objective_value(::Model)` to get the objective value
- Use `value(::VariableRef)` to get the primal value of a variable
- Use `dual(::ConstraintRef)` to get the value of the dual variable associated with a constraint

In [72]:
x_value = value(x)
obj_value = objective_value(model)

println("Termination status: ", termination_status(model))
println("Primal status:      ", primal_status(model))
println("Dual status:        ", dual_status(model))
println("      x | $(x_value)")
println("    π/2 | $(π/2)")
println("--------+----------------------")
println("cos²(x) | $(obj_value)")

Termination status: LOCALLY_SOLVED
Primal status:      FEASIBLE_POINT
Dual status:        FEASIBLE_POINT
      x | 1.5707963267948966
    π/2 | 1.5707963267948966
--------+----------------------
cos²(x) | 3.749399456654644e-33


## Example: Simple Economic Dispatch

*This example is adapted from a [tutorial given at the 2015 Grid Science Winter School](https://github.com/JuliaOpt/juliaopt-notebooks/blob/3110eddaf5effdecfee687739295bea05731ba33/notebooks/Dvorkin%20-%20Power%20systems%20-%20Economic%20dispatch%20and%20Unit%20commitment.ipynb).*

Economic dispatch (ED) is an optimization problem that minimizes the cost of supplying energy demand subject to operational constraints on power system assets. In its simplest instantiation, ED is a Linear Programing (LP) problem solved for an aggregated load and wind forecast and for a single infinitesimal moment. Mathematically, the ED problem can be written as follows:
$$
\min \sum_{i \in I} c^g_{i} \cdot g_{i} + c^w \cdot w,
$$
where $c_{i}$ and $g_{i}$ are the incremental cost ($\$/MWh$) and power output ($MW$) of the $i^{th}$ thermal generator, respectively, and $c^w$ and $w$ are the incremental cost ($\$/MWh$) and wind power injection ($MW$), respectively.

s.t.

<li> Minimum ($g^{\min}$) and maximum ($g^{\max}$) limits on power outputs of the thermal generators: </li>
$$
g^{\min}_{i} \leq g_{i} \leq g^{\max}_{i}.
$$
<li>Constraint on the wind power injection:</li>
$$
0 \leq w \leq w^f, 
$$
where $w$ and $w^f$ are the wind power injection and wind power forecast, respectively.

<li>Power balance constraint:</li>
$$
\sum_{i \in I} g_{i} + w = d, 
$$
where $d$ is the demand.

Further reading on ED models can be found in A. J. Wood, B. F. Wollenberg, and G. B. Sheblé, "Power Generation, Operation and Control", Wiley, 2013.

### JuMP Implementation of Economic Dispatch 

First, we need to load some plotting packages.  Note, compilation may take a few minutes.

In [73]:
using Interact
using Plots

Now, we define some problem data for a model with two thermal generators and one wind generator.

In [74]:
# Define some input data about the test system
# Maximum power output of generators
const GENERATION_MAX = [1000, 1000]
# Minimum power output of generators
const GENERATION_MIN = [0, 300]
# Incremental cost of generators 
const COST_GENERATION = [50, 100]
# Incremental cost of wind generators
const COST_WIND = 50
# Total demand
const DEMAND = 1500
# Wind forecast
const WIND_MAX = 200;

In the next cell, we create a Julia function that formulates and solves the economic dispatch problem.

In [75]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)

    economic_dispatch = Model(GLPK.Optimizer)
    
    # Define decision variables    
    @variables(economic_dispatch, begin
        g[i=1:2]  # Thermal generation (MW).
        w >= 0  # Wind power (MW).
    end)

    # Define the objective function
    @objective(economic_dispatch, Min, 
        sum(cost_of_thermal[i] * g[i] for i in 1:2) + cost_of_wind * w
    )

    # Define the constraint on the maximum and minimum power output of each generator.
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end
    
    @constraints(economic_dispatch, begin
        # Define the constraint on the wind power injection
        w <= WIND_MAX
        # Define the power balance constraint
        sum(g[i] for i in 1:2) + w == DEMAND
    end)

    # Solve statement
    optimize!(economic_dispatch)
    
    # Return the optimal value of the objective function and its minimizers
    # as a NamedTuple.
    return (
        generation = value.(g), 
        wind_generation = value(w),
        wind_spillage = WIND_MAX - value(w),
        cost = objective_value(economic_dispatch)
    )
end

# Solve the economic dispatch problem
solution = solve_economic_dispatch()

println("Dispatch")
println("   Generators: ", solution.generation, " MW")
println("         Wind: ", solution.wind_generation, " MW")
println("Wind spillage: ", solution.wind_spillage, " MW") 
println("----------------------------------")
println("Total cost: \$", solution.cost)  

Dispatch
   Generators: [1000.0, 300.0] MW
         Wind: 200.0 MW
Wind spillage: 0.0 MW
----------------------------------
Total cost: $90000.0


### Economic dispatch with adjustable incremental costs

In the following exercise we introduce a manipulator to vary the cost of wind generation and observe its impact the total cost, dispatch of generators G1 and G2, utilization of available wind under different values of the incremental cost of generator G1.

In [76]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.01:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 26,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
13, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000122fe2d10, Task (runnable) @0x0000000122fe2d10), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"50.0\",\"55.0\",\"60.0\",\"65.0\",\"70.0\",\"75.0\",\"80.0\",\"85.0\",\"90.0\",\"95.0\",\"100.0\",\"105.0\",\"110.0\",\"115.0\",\"120.0\",\"125.0\",\"130.0\",\"135.0\",\"140.0\",\"145.0\",\"150.0\",\"155.0\",\"160.0\",\"16

## Nonlinear example

JuMP can also be used to solve non-linear problems (NLP). We saw a brief hint of this earlier when we used Ipopt and the `@NLobjective` macro. In the next example, we add nonlinearity to the cost of the second generator. The new cost of generation is
$$\text{generation_cost} = c_1^g\times g_1 + \frac{c_2^g\times  g_2^{1.5}}{\sqrt{1000}}.$$


In [77]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_nonlinear_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)
    economic_dispatch = Model(Ipopt.Optimizer)
    set_optimizer_attribute(economic_dispatch, "print_level", 0)
    
    @variables(economic_dispatch, begin
        g[i=1:2] >= 0
        w >= 0
    end)

    # ===========================================================
    # You can write out nonlinear expression in the @NLobjective macro
    # The same also applies for @NLconstraint.
    @NLobjective(economic_dispatch, Min,
        cost_of_thermal[1] * g[1] + 
        cost_of_thermal[2] * g[2]^1.5 / sqrt(1000) + 
        cost_of_wind * w)
    # ===========================================================
    # Look! This bit changed.
    function generator_cost(g1, g2)
        return cost_of_thermal[1] * g1 + cost_of_thermal[2] * g2^1.5 / sqrt(1000)
    end
    JuMP.register(economic_dispatch, :generator_cost, 2, generator_cost, autodiff=true)
    @NLobjective(economic_dispatch, Min, 
        generator_cost(g[1], g[2]) + cost_of_wind * w)
    # ===========================================================
    
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end    
    @constraints(economic_dispatch, begin
        w <= WIND_MAX
        sum(g[i] for i in 1:2) + w == DEMAND
    end)
    optimize!(economic_dispatch)
    return (
        generation = value.(g), 
        wind_generation = value(w),
        wind_spillage = WIND_MAX - value(w),
        cost = objective_value(economic_dispatch)
    )
end

solve_nonlinear_economic_dispatch

In [78]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.2:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.1:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_nonlinear_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 13,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
7, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/carleton/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/carleton/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/carleton/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/carleton/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/carleton/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000110069f90, Task (runnable) @0x0000000110069f90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"50.0\",\"60.0\",\"70.0\",\"80.0\",\"90.0\",\"100.0\",\"110.0\",\"120.0\",\"130.0\",\"140.0\",\"150.0\",\"160.0\",\"170.0\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"4738911583611541819\",\"id\":\"ob_57\",\